In [ ]:
import tensorflow as tf
import tensorflow.python.platform.build_info as build

tf.__version__

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(f'Default GPU Device:{tf.test.gpu_device_name()}')

In [ ]:
print(build.build_info['cuda_version'])
print(build.build_info['cudnn_version'])

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt

import plotly
import plotly.graph_objs as go

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle as p

to_categorical  = tf.keras.utils.to_categorical
mnist           = tf.keras.datasets.mnist
load_model      = tf.keras.models.load_model
model_from_json = tf.keras.models.model_from_json

In [ ]:
#pip install plotly

In [ ]:
batch_size = 128
num_classes = 10
epochs = 5

img_rows, img_cols = 28, 28

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
print(x_train.shape, x_test.shape)

In [ ]:
y_train[0:10]

In [ ]:
plt.figure(figsize=(5, 4))
for i in range(20):
    plt.subplot(4, 5, i+1)
    plt.imshow(x_train[i], cmap='gray')
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()
plt.savefig('train_mnist.png', dpi=300)

In [ ]:
x_train.min(), x_train.max()

In [ ]:

temp_x_test = x_test

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print(x_train.min(), x_train.max())

In [ ]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
#Feature Extraction
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

#Image Classification
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
start = time.time()

his = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

done = time.time()
print(done - start)

In [ ]:
filepath_history_model = 'history_model'

with open(filepath_history_model, 'wb') as file:
    p.dump(his.history, file)

In [ ]:
filepath='model.h5'
model.save(filepath)

In [ ]:
filepath_model = 'model.json'
filepath_weights = 'model.weights.h5'

model_json = model.to_json()
with open(filepath_model, "w") as json_file:
    json_file.write(model_json)
    model.save_weights(filepath_weights)

In [ ]:
with open(filepath_history_model, 'rb') as file:
    his = p.load(file)

In [ ]:
h1 = go.Scatter(y=his['loss'],
                    mode="lines", line=dict(
                    width=2,
                    color='blue'),
                    name="loss"
                   )
h2 = go.Scatter(y=his['val_loss'],
                    mode="lines", line=dict(
                    width=2,
                    color='red'),
                    name="val_loss"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data, layout=layout1)
plotly.offline.iplot(fig1, filename="testMNIST")

In [ ]:
predict_model = load_model(filepath)
predict_model.summary()

In [ ]:
with open(filepath_model, 'r') as f:
    loaded_model_json = f.read()
    predict_model = model_from_json(loaded_model_json)
    predict_model.load_weights(filepath_weights)

In [ ]:
predict_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

predict_model.summary()

In [ ]:
plt.figure(figsize=(5, 4))
for i in range(20):
    plt.subplot(4, 5, i+1)
    plt.imshow(temp_x_test[i], cmap='gray')

plt.tight_layout()
plt.savefig('test_mnist.png', dpi=300)

In [ ]:
result = predict_model.predict(x_test[:1])
result = np.argmax(result, axis=-1)

print(result)

In [ ]:
score = predict_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
predicted_classes = predict_model.predict(x_test)
predicted_classes = np.argmax(predicted_classes,axis=-1)

In [ ]:
y_true = np.argmax(y_test, axis=-1)

In [ ]:
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]

print("Correct predicted classes:",correct.shape[0])
print("Incorrect predicted classes:",incorrect.shape[0])

In [ ]:
confusion_matrix(y_true, predicted_classes)

In [ ]:
labels = {0 : "Zero", 1: "One", 2: "Two", 3: "Three", 4: "Four",
          5: "Five", 6: "Six", 7: "Seven", 8: "Eight", 9: "Nine"}

target_names = ["Class {} ({}) :".format(i,labels[i]) for i in range(num_classes)]
print(classification_report(y_true, predicted_classes, target_names=target_names, digits=4))